Ссылка на датасет - https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?resource=download

Цель - Предсказание стоимости автомобилей c помощью библиотеки CatBoost. 

Метрики регрессии - MAE, MAPE

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from catboost import CatBoostRegressor


In [3]:
df = pd.read_csv('archive/bmw.csv')


In [108]:
df.head()


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


Разделим данные на обучающую(train - 60%) и тестовую(val - 20%) и валидационную(20%) выборки

На обучающей выборке(train) мы будем обучаться;

На валидационной выборке(val) на каждом шаге обучения считать ошибку и смотреть как она изменяется, чтобы найти точку с наилучшим качеством(наименьшей ошибкой);

На тестовой(test) будем проверять работу модели на данных, которые модель не видела, чтобы иметь ожидание ошибки, которую модель будет давать на новых данных

In [8]:
train, test = train_test_split(df,train_size=0.6, random_state=42)


In [9]:
len(train)/len(df) #На train приходится 60% данных


0.5999443465355718

In [109]:
len(test)/len(df) # на test приходится 40% данных


0.20007420461923756

Теперь 50% test выделим на валидационную выборку(val)

In [11]:
val, test = train_test_split(test,train_size=0.5, random_state=42)


In [12]:
len(val)/len(df)


0.19998144884519062

In [13]:
len(test)/len(df)


0.20007420461923756

Теперь тестовая и валидационная выборки составляют по 20% от данных

Сначала запустим catboost на 3-х фичах(год выпуска, тип трансмиссии, объем двигателя)

Необходимо указать столбцы, в которых находится текст - категориальные фичи(transmission). 
Catboost преобразует категориальные фичи в числовые признаки.

In [66]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']


X - признаки(фичи), y - целевой признак(таргет), cat_features - категориальные фичи

Создаем модель, в качестве метрики будем использовать mean_absolute_percentage_error(MAPE)

In [67]:
model = CatBoostRegressor(cat_features=cat_features,
                          eval_metric='MAPE',
                          learning_rate=0.03,
                          random_seed=42,
                          verbose = 100)


Запускаем обучение

In [68]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))


0:	learn: 0.4746987	test: 0.4699730	best: 0.4699730 (0)	total: 4.52ms	remaining: 4.52s
100:	learn: 0.1681230	test: 0.1638970	best: 0.1638970 (100)	total: 120ms	remaining: 1.07s
200:	learn: 0.1561398	test: 0.1538311	best: 0.1538311 (200)	total: 217ms	remaining: 864ms
300:	learn: 0.1543096	test: 0.1525317	best: 0.1525309 (299)	total: 302ms	remaining: 701ms
400:	learn: 0.1534465	test: 0.1519221	best: 0.1519198 (397)	total: 383ms	remaining: 572ms
500:	learn: 0.1527824	test: 0.1516330	best: 0.1516164 (480)	total: 474ms	remaining: 472ms
600:	learn: 0.1522590	test: 0.1514284	best: 0.1514284 (600)	total: 573ms	remaining: 381ms
700:	learn: 0.1518052	test: 0.1513704	best: 0.1513673 (689)	total: 672ms	remaining: 287ms
800:	learn: 0.1514495	test: 0.1513967	best: 0.1513651 (703)	total: 774ms	remaining: 192ms
900:	learn: 0.1510958	test: 0.1513657	best: 0.1513489 (873)	total: 871ms	remaining: 95.7ms
999:	learn: 0.1508297	test: 0.1513297	best: 0.1513168 (992)	total: 975ms	remaining: 0us

bestTest = 0.

На валидационых данных мы получили MAPE - 15.1%

Теперь посмотрим на test

In [69]:
model.predict(test[X])


array([45427.60783889, 21777.25462924, 16183.61825252, ...,
       28427.4725127 , 16183.61825252, 27791.4679527 ])

Добавим предикт в test для расчета MAPE

In [70]:
test['price_pred'] = model.predict(test[X])


Напишем функцию для удобства расчета MAE и MAPE

In [71]:
def error(y_true, y_pred):
    print(mean_absolute_error(y_true, y_pred))
    print(mean_absolute_percentage_error(y_true, y_pred))


In [72]:
error(test['price'], test['price_pred'])


3360.896182174203
0.15296695559222623


На test мы получили MAPE - 15.3% 

Теперь обучим модель на всех фичах

In [83]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']


In [84]:
parameters = {'cat_features' : cat_features,
              'eval_metric' : 'MAPE',
              'learning_rate' : 0.08,
              'random_seed' : 42,
              'verbose' : 100}


In [85]:
model = CatBoostRegressor(**parameters)


In [86]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))


0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 5.52ms	remaining: 5.51s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 187ms	remaining: 1.66s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 359ms	remaining: 1.43s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 522ms	remaining: 1.21s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 694ms	remaining: 1.03s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 870ms	remaining: 867ms
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.04s	remaining: 693ms
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 1.22s	remaining: 520ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 1.39s	remaining: 346ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 1.56s	remaining: 172ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 1.74s	remaining: 0us

bestTest = 0.0

На валидационых данных мы получили MAPE - 6.9%

Посмотрим результат на test

In [58]:
test['price_pred_all'] = model.predict(test[X])


In [59]:
error(test['price'], test['price_pred_all'])


1557.1393226999464
0.07360851115958396


На test мы получили MAPE - 7.4%

Обучим модель на всех данных

Покажем модели больше данных, объединив обучающую и валидационную выборки с помощью concat

In [90]:
train_full = pd.concat([train,val])


In [91]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']


In [92]:
parameters = {'cat_features' : cat_features,
              'eval_metric' : 'MAPE',
              'learning_rate' : 0.08,
              'random_seed' : 42,
              'verbose' : 100}


In [93]:
model = CatBoostRegressor(**parameters)


In [94]:
model.fit(train[X], train[y], eval_set=(val[X],val[y]))


0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 4.09ms	remaining: 4.09s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 175ms	remaining: 1.56s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 350ms	remaining: 1.39s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 517ms	remaining: 1.2s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 694ms	remaining: 1.04s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 874ms	remaining: 870ms
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.05s	remaining: 696ms
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 1.23s	remaining: 523ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 1.41s	remaining: 350ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 1.58s	remaining: 174ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 1.76s	remaining: 0us

bestTest = 0.06

Создадим новую модель и зададим такое же количество итераций(994)

In [97]:
parameters = {'iterations' : model.best_iteration_ + 1 ,
              'cat_features' : cat_features,
              'eval_metric' : 'MAPE',
              'learning_rate' : 0.08,
              'random_seed' : 42,
              'verbose' : 100}


In [98]:
model = CatBoostRegressor(**parameters)


In [99]:
model.fit(train_full[X],train_full[y])


0:	learn: 0.4538016	total: 4.78ms	remaining: 4.75s
100:	learn: 0.0854178	total: 215ms	remaining: 1.9s
200:	learn: 0.0745022	total: 416ms	remaining: 1.64s
300:	learn: 0.0701717	total: 608ms	remaining: 1.4s
400:	learn: 0.0676577	total: 813ms	remaining: 1.2s
500:	learn: 0.0656926	total: 1s	remaining: 990ms
600:	learn: 0.0638104	total: 1.2s	remaining: 787ms
700:	learn: 0.0624302	total: 1.4s	remaining: 585ms
800:	learn: 0.0612275	total: 1.61s	remaining: 388ms
900:	learn: 0.0599823	total: 1.81s	remaining: 186ms
993:	learn: 0.0590330	total: 2s	remaining: 0us


Теперь сделаем предикт на test

In [101]:
test['price_pred_all_features_and_data'] = model.predict(test[X])


In [107]:
error(test['price'], test['price_pred_all']) # Результат на всех фичах


1557.1393226999464
0.07360851115958396


In [105]:
error(test['price'], test['price_pred_all_features_and_data']) # Результат с учетом объединения train и val


1512.6160783572977
0.07305203023986592


MAPE на test c учетом всех данных - 7.3%. 

Объединив обучающую и валидационную выборки мы еще немного улучшили результат.